# Function to complete exsting JSON files with the neighborhood's description based on ChatGPT answers.

# **Warning: to run this notebook you need to have a OpenAI API key!**

# Dependencies

In [6]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import json
import os

# Load JSON files to be updated

In [28]:
def load_data(path):
    try: 
        with open(path, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"An error as occured: {e}")
        
def get_file_names(directory):
    return [
        os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.json')
    ]

In [29]:
directory = r"../data/google_data_isochrone_pop_cgpt"
FILES = get_file_names(directory)
FILES

['../data/google_data_isochrone_pop_cgpt\\Ex1_8004_Zurich_Werdgartengasse_4.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex2_3027_Bern_Colombstrasse_39.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex3_1006_Lausanne_Av_d_Ouchy_58.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex4_8355_Aadorf_Bruggwiesenstrasse_5.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex5_6319_Allenwinden_Winzruti_39.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex6_8005_Zurich_Heinrichstrasse_200.json',
 '../data/google_data_isochrone_pop_cgpt\\Ex7_8003_Zurich_Birmensdorferstrasse_108.json']

# Helper functions

## Create dataframe of facilities to pass to ChatGPT

In [30]:
def list_facilities(data):
    
    facilities_list={}

    for facility_type in data['facilities'].keys():
        facilities_list[facility_type] = {}
        facilities_list[facility_type]['count'] = data['facilities'][facility_type]['count']
        if 'closest' in data['facilities'][facility_type].keys():
            facilities_list[facility_type]['closest'] = data['facilities'][facility_type]['closest']['name']
            facilities_list[facility_type]['walking_time'] = data['facilities'][facility_type]['closest']['travel_time']
        else:
            facilities_list[facility_type]['closest'] = 'none'
            facilities_list[facility_type]['walking_time'] = 'none'

    return pd.DataFrame.from_dict(facilities_list, orient='index')


In [31]:
# Adjust pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1)
pd.set_option('display.width', 1000)

for file in FILES:
    data = load_data(file)
    print(data['original_address']['address'])
    print(list_facilities(data))
    print('------')


8004 Zürich, Werdgartengasse 4
                             count                           closest walking_time
bars                         4      Certo                             5 mins     
restaurants                  9      Certo                             5 mins     
kindergarten                 6      Saint Georges Hotel               3 mins     
public_transportation        34     Saint Georges Hotel               3 mins     
gym_fitness                  14     Saint Georges Hotel               3 mins     
grocery_stores_supermarkets  14     Coop Pronto Shop Zürich Wiedikon  2 mins     
gas_ev_charging              4      Saint Georges Hotel               3 mins     
schools                      6      Bibi Bachtadze Group AG           2 mins     
------
3027 Bern, Colombstrasse 39
                             count                                    closest walking_time
bars                         4      Bits & Bites Bern                          5 mins     
restaurants   

## Create Prompt for ChatGPT

In [32]:
def summarize_neigborhood(FILE, style:str):
    # load data
    data = load_data(FILE)
    facilities_df = list_facilities(data)

    # Define the message
    messages = [
    {"role": "system", "content": """
    You are a data scientist. 
    Your task is to define the atmosphere/vibe of a given neighborhood. 
    A neighborhood is defined as an area reachable with a 10min foot walk around a given real estate property. 
    """},
    {"role": "user", "content": f"""
    --------
    TASK: 
    the DATA given below lists the location, the number of inhabitants as well as the facilities found within 10 min walking distance of the location. 
    For each facility type the count of facilities and the walking time to the closest facility (including its name) are indicated. 
    Considering ALL of this information, return 3 sentences describing the atmosphere/vibe of this neighborhood in the style indicated below. 
    In your anser, do NOT repeat the address of the location and do NOT list all the facilites given in "Data".
    --------
    STYLE: "{style}"
    --------
    DATA:
    Location: \'{data['original_address']['address']}\'.
    Number of inhabitants: about \'{str(data['population']['total_pop'])}\' permanent residents in the neighborhood (data from 2022).
    List of facilities to be found within 10 min walking distance: 
    \'{facilities_df}\'.
    """}
    ]

    # Call the OpenAI API with the chat completion endpoint
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0125",  # advice from Adriano, instead of gpt-3.5-turbo (because of tokens usage)
    messages=messages,
    max_tokens=1000,  # Adjust 
    n=1,
    stop=None,
    temperature=0  # Setting temperature to 0 for deterministic results
    )

    # Extract the classification result from the response
    result = response.choices[0].message['content'].strip()

    return result


# Load ChatGPT credentials

In [33]:
import locale, yaml, openai

locale.getpreferredencoding = lambda: "UTF-8"

with open('credentials.json', 'r') as file:
    api_creds = yaml.safe_load(file)

os.environ['OPENAI_API_KEY'] = api_creds['openai_key']

openai.api_key = os.environ['OPENAI_API_KEY']

# Function to add ChatGPT answers to JSON file

In [34]:
def add_text_description_to_existing_JSONfile(path, styles=['Lex Fridman', 'Real estate agent', 'neutral without emphasis']):
    """Adds ChatGPT_answer to an existing json file.
    Arg:
    - path : path incl. extension ".json"
    - styles: a list of strings describing the style of the answer
    """
    # loads the json file
    with open(path, 'r') as file:
        data = json.load(file)
    if 'text_description' in data.keys():
        print(path +' : \nThis JSON file already has a text description.')
        return
    else:
        # appends the description  to the existing dictionary, but initialized as an empty dict (since we can pass several STYLES)
        data['text_description'] = {}

        for style in styles:
            text = summarize_neigborhood(path, style)
            data['text_description'][style] = {'Style': style, 'text': text}

        # overwrites the file:
        with open(path, 'w') as file:
            json.dump(data, file, indent=2) # indent = 2 since in the highest level    

## update files

In [35]:
for file in FILES:
    add_text_description_to_existing_JSONfile(file, styles=['Lex Fridman', 'Real estate agent', 'neutral without emphasis'])

../data/google_data_isochrone_pop_cgpt\Ex1_8004_Zurich_Werdgartengasse_4.json : 
This JSON file already has a text description.
../data/google_data_isochrone_pop_cgpt\Ex2_3027_Bern_Colombstrasse_39.json : 
This JSON file already has a text description.
../data/google_data_isochrone_pop_cgpt\Ex3_1006_Lausanne_Av_d_Ouchy_58.json : 
This JSON file already has a text description.
../data/google_data_isochrone_pop_cgpt\Ex4_8355_Aadorf_Bruggwiesenstrasse_5.json : 
This JSON file already has a text description.
../data/google_data_isochrone_pop_cgpt\Ex5_6319_Allenwinden_Winzruti_39.json : 
This JSON file already has a text description.
../data/google_data_isochrone_pop_cgpt\Ex6_8005_Zurich_Heinrichstrasse_200.json : 
This JSON file already has a text description.
../data/google_data_isochrone_pop_cgpt\Ex7_8003_Zurich_Birmensdorferstrasse_108.json : 
This JSON file already has a text description.


# Examples:
### other style (neutral)

Prompt:

    You are a data scientist. 
    Your task is to define the atmosphere/vibe of a given neighborhood. 
    A neighborhood is defined as an area reachable with a 10min foot walk around a given real estate property. 
    
    --------
    Task: taking into consideration the location, the number of inhabitants as well as the number of the different facilities and the walking time to the closest facility, return 3 sentences describing the atmosphere/vibe of this neighborhood in the indicated style. 
    --------
    Style: "Neutral, without emphasis"
    --------
    Data:
    Real estate property address: '1006 Lausanne, Av. d'Ouchy 58'.
    Population in neighborhood: about '9994' permanent residents (data from 2022).
    List of facilities to be found within 10 min walking distance: 
    '                             count  \
bars                            12   
restaurants                     12   
kindergarten                    22   
public_transportation           28   
gym_fitness                     15   
grocery_stores_supermarkets     15   
gas_ev_charging                 14   
schools                          5   

                                                              closest_facility  \
bars                                                               White Horse   
restaurants                                   Takayama. Sushi bar & restaurant   
kindergarten                                                  ImmoStreet.ch SA   
public_transportation                                         ImmoStreet.ch SA   
gym_fitness                                                   ImmoStreet.ch SA   
grocery_stores_supermarkets                                   ImmoStreet.ch SA   
gas_ev_charging                                               ImmoStreet.ch SA   
schools                      Formasuisse, Formations Rh, Management, Certif...   

                            walking_time_to_closest_facility  
bars                                                   1 min  
restaurants                                            1 min  
kindergarten                                           1 min  
public_transportation                                  1 min  
gym_fitness                                            1 min  
grocery_stores_supermarkets                            1 min  
gas_ev_charging                                        1 min  
schools                                               2 mins  '.
    --------
Answer: In the neighborhood around '1006 Lausanne, Av. d'Ouchy 58', with approximately 9994 permanent residents, there are various facilities such as bars, restaurants, kindergarten, public transportation, gym/fitness centers, grocery stores/supermarkets, gas/EV charging stations, and schools within a 10-minute walking distance. The closest facilities include White Horse bar, Takayama Sushi bar & restaurant, ImmoStreet.ch SA kindergarten, and public transportation, all reachable within 1-2 minutes on foot. The atmosphere in this neighborhood can be described as bustling with a diverse range of amenities catering to the needs of the local community.

### first example wit Lex friedman
Example just to make sure the prompting is correct: it is.

Prompt:

    You are a data scientist. 
    Your task is to define the atmosphere/vibe of a given neighborhood. 
    A neighborhood is defined as an area reachable with a 10min foot walk around a given real estate property. 
    
    Data:
    ------
    Real estate property address: '1006 Lausanne, Av. d'Ouchy 58'.
    Population in neighborhood: about '9994'. permanent residents (data from 2022).
    List of facilities to be found within 10 min walking distance: 
    '                             count  \
bars                            12   
restaurants                     12   
kindergarten                    22   
public_transportation           28   
gym_fitness                     15   
grocery_stores_supermarkets     15   
gas_ev_charging                 14   
schools                          5   

                                                              closest_facility  \
bars                                                               White Horse   
restaurants                                   Takayama. Sushi bar & restaurant   
kindergarten                                                  ImmoStreet.ch SA   
public_transportation                                         ImmoStreet.ch SA   
gym_fitness                                                   ImmoStreet.ch SA   
grocery_stores_supermarkets                                   ImmoStreet.ch SA   
gas_ev_charging                                               ImmoStreet.ch SA   
schools                      Formasuisse, Formations Rh, Management, Certif...   

                            walking_time_to_closest_facility  
bars                                                   1 min  
restaurants                                            1 min  
kindergarten                                           1 min  
public_transportation                                  1 min  
gym_fitness                                            1 min  
grocery_stores_supermarkets                            1 min  
gas_ev_charging                                        1 min  
schools                                               2 mins  '.
    ----
    Task: taking into consideration the location, the number of inhabitants as well as the number of the different facilities and the walking time to the closest facility, return:
    - 5 key words describing the atmosphere/vibe of this neighborhood.
    - 3 sentences describing the atmosphere/vibe of this neighborhood in the "Lex Friedman style"
    --------
Answer: **Key Words Describing the Atmosphere/Vibe:**
1. Vibrant
2. Diverse
3. Convenient
4. Active
5. Community-oriented

**Atmosphere/Vibe Description in the "Lex Friedman Style":**
In the heart of Av. d'Ouchy, this neighborhood pulses with a vibrant energy, fueled by a diverse community of approximately 9994 residents. Within a 10-minute stroll, one can easily find themselves immersed in a tapestry of convenience, from a plethora of dining options to top-notch fitness facilities. The streets hum with activity, echoing the harmonious blend of bustling public transportation and the tranquility of nearby kindergartens and schools, creating a dynamic and community-oriented ambiance that beckons both locals and visitors alike.
Prompt:

### with more restrictions on the answer:

Prompt:

    You are a data scientist. 
    Your task is to define the atmosphere/vibe of a given neighborhood. 
    A neighborhood is defined as an area reachable with a 10min foot walk around a given real estate property. 
    
    --------
    TASK: 
    the DATA given below lists the location, the number of inhabitants as well as the facilities found within 10 min walking distance of the location. 
    For each facility type the count of facilities and the walking time to the closest facility (including its name) are indicated. 
    Considering ALL of this information, return 3 sentences describing the atmosphere/vibe of this neighborhood in the style indicated below. 
    In your anser, do NOT repeat the address of the location and do NOT list all the facilites given in "Data".
    --------
    STYLE: "Neutral, without emphasis"
    --------
    DATA:
    Location: '1006 Lausanne, Av. d'Ouchy 58'.
    Number of inhabitants: about '9994' permanent residents in the neighborhood (data from 2022).
    List of facilities to be found within 10 min walking distance: 
    '                             count  \
bars                            12   
restaurants                     12   
kindergarten                    22   
public_transportation           28   
gym_fitness                     15   
...
gas_ev_charging                                        1 min  
schools                                               2 mins  '.
    --------
Answer: The neighborhood exudes a bustling yet relaxed ambiance with a variety of amenities within easy reach, including numerous bars, restaurants, and fitness centers. Residents benefit from a diverse community with a significant number of schools and kindergartens nearby, fostering a family-friendly environment. The convenience of public transportation and grocery stores further enhances the neighborhood's livability, creating a well-connected and vibrant atmosphere.
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...

# END OF NOTEBOOK